In [537]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

str_parser = StrOutputParser()

### Step 1 -> Indexing

In [538]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

video_ids = "kFesG4KC6AA"
print(f"Fetching transcript for video: {video_ids}")

try:
    transcript_list = YouTubeTranscriptApi()
      
    fetched_transcript = transcript_list.fetch(video_ids, languages=['hi', 'en'])
 
    transcript_text = " ".join([chunk.text for chunk in fetched_transcript])
    
    print("\nTranscript fetched successfully!")
    print(transcript_text[:500] + "...") 
    

except TranscriptsDisabled:
    print("Transcripts are disabled for this video.")
except NoTranscriptFound:
    print("No transcript found for the requested languages.")
except Exception as e:
    print(f"An error occurred: {e}")

Fetching transcript for video: kFesG4KC6AA

Transcript fetched successfully!
एआई वेव आ गई है। YouTube पे तो हर कोई बिलिनेियर बन चुका है। लाइक इन वन मंथ वो इतना इतने पैसे कमा चुके हैं। ऐसे हो चुके हैं। अगर मुझे मैक्सिमम अमाउंट ऑफ पैसे बनाने हैं इस एआई वेव को यूज़ करके और मेरे पास कोई रूल लाइक क्लोज टू ज़ीरो रूल्स हैं। एंड इफ यू हैड टू डू दैट व्हाट वुड यू डू? द मोर यू थ्रो कंप्यूट एट इट द बेटर दी आई बिकम। दिस हैज़ बीन ट्रू लार्जली। बट कह रहे हैं कि इसकी लिमिट्स आना शुरू हो गई है। राइट? तो अगर मैं एक एज अ एज अ फ्रेश इंजीनियर नेक्स्ट ईयर ग्रेजुएट कर रहा हूं। हम्म वी डोंट हैव द ...


In [539]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = splitter.create_documents([transcript_text])

In [540]:
# len(chunks)

In [541]:
# chunks[0]

In [542]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embeddings)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 561.21it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [543]:
# Get all the document IDs stored in the vectorstore
stored_ids = vectorstore.get()['ids']
print(f"Total documents: {len(stored_ids)}")
print(f"Document IDs: {stored_ids}")

Total documents: 16040
Document IDs: ['bdafaeff-c5f5-45ea-9a6f-c684cab91ef8', 'e1394bb6-c2e7-4655-b20c-29d269309d54', '9f8d8e71-28a9-43ca-8096-88ec677dae30', '9b098c4e-4cb3-4667-9e70-7e807ef52c85', '7ded2068-295a-45c0-a0bf-b508d6e5eb78', 'd57b5f07-90c6-41eb-a450-47892d56c54c', '56eb20ff-3bc4-40fc-94db-db6df1aab0c0', '3f2ddb06-a8bc-4665-948b-dd597df5dfde', 'ed74c40e-facf-4d96-b49c-0bc32b9ca399', '2e8c181d-288d-4ecd-b045-f7ccf909113e', '2f595a83-18cf-4019-9cdf-c02a21de0286', 'cc3051d6-67bb-44aa-a092-9169c771fe7e', '9002bc47-9496-4b48-838d-b0c0facfb3b3', 'be455c20-456b-4a27-9106-051488d4e0c4', 'aabfa045-baac-4ce1-9ab0-9462b4120d9d', '76ad23db-222c-4ca2-a657-be91a10ccd62', '3ced2f3a-0840-444a-ad76-f6809a8ea021', 'a92de618-bfc4-4559-aed0-41bffba9db32', 'c8fc8433-62f6-41ad-9554-e051a3427e00', '751061d4-6a9b-4935-94ea-60e64e214a89', 'ef912a0f-75e8-4cbe-8f45-b2ab627d4676', '8598f8d0-7cd9-42a0-82d0-beeb81fac09d', '4d3c7124-cce5-4606-ae4c-b2252544c297', '78396ce0-76fd-42c8-8381-e0d900698292', '2

In [544]:
id = stored_ids[0]
vectorstore.get(ids=[id])

{'ids': ['bdafaeff-c5f5-45ea-9a6f-c684cab91ef8'],
 'embeddings': None,
 'documents': ['एआई वेव आ गई है। YouTube पे तो हर कोई बिलिनेियर बन चुका है। लाइक इन वन मंथ वो इतना इतने पैसे कमा'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None]}

### Step 2 -> Retriver

In [545]:
# Reduced k from 500 to 150. 
# Sending 500 chunks creates a prompt that is too large for the inference API, causing the 400 Bad Request error.
retriver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 150})
retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000021C8FC8F440>, search_kwargs={'k': 150})

In [546]:
# reply = retriver.invoke("What is the video about?")
# for item in reply:
#     print(item.page_content)

### Step 3 -> Augmentation

In [547]:
api_key = os.getenv("HUGGINGFACE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key,
)

model = ChatHuggingFace(llm=llm, temperature=0.7)

In [548]:
prompt = PromptTemplate(
    template = '''You are a precise and helpful AI Assistant.
    Your task is to answer the [Question] using ONLY the provided [Context], must reply in English or Roman English language. 

                    Rules to follow:
                    1. Use the [Context] provided below to formulate your answer.
                    2. If the answer is not contained within the [Context], strictly state: "I'm sorry, but I don't have enough information to answer this." 
                    3. Do not use any outside knowledge or prior training to answer.
                    4. Language: Must Respond in English or Roman English, matching the user's style, no matters in which language you recive context...
                    5. Keep the response relevant to context.

                    Context:
                    {context}

                    Question:
                    {question}

                    Answer: You are a translation expert. Translate the following text into English or Roman English. 
        If the Answer is already in English, return it as is.
        Respond with ONLY the translated text, nothing else.''',
    input_variables=["context", "question"]
)

In [549]:
question = "(The Future of voice AI) isss video ma kya baat howi ha iska releated wo batao muja?"
retrived_context = retriver.invoke(question)

In [550]:
# context_text = "\n".join([item.page_content for item in retrived_context])

In [551]:
# print(context_text)

In [552]:
# final_prompt = prompt.invoke({"context": context_text, "question": question})

### Step 4 -> Generation

In [553]:
# answer = model.invoke(final_prompt)
# print("Answer:", answer.content)

## Building Through Chains

1) ### Parallel Chain

In [563]:
def formating(retrived_context):
    context_text = "\n".join([item.page_content for item in retrived_context])
    return context_text

In [564]:
parallel_chain = RunnableParallel(
    {"context": retriver | RunnableLambda(formating),
     "question": RunnablePassthrough()}
)

In [565]:
parallel_chain.invoke(question)

{'context': 'और सारा काम करना है बट इफ यू कुड फॉर एग्जांपल हैव\nकंपनी में फला बंदा काम करता है हु इज़ द सीईओ क्या\nकिमी की तरफ से आ गया है या किम के2 आ गया या फिर\nआर सम इंडस्ट्रीज जो के आर नेगलेक्टेड आर मे बी ऑन\nमें आप काम करते हैं। यू सॉफ्टवेयर डिज़ाइन\nपॉइंट। अ आज हम स्पेसिफिकली बात करना, मैं\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द फॉलोइंग। आज अगर आपने अपना कोई एप्लीकेशन बनाता है आप उसका एपीआई सर्वर बनाते हैं।\nबट द आईडिया इज़ द 

2) ### Main Chain

In [567]:
main_chain = (
    parallel_chain
    | prompt
    | model
    | str_parser
)

results = main_chain.invoke(question)
print("Final Answer:", results)

Final Answer: The Future of voice AI isss video ma kya baat howi ha iska releated wo batao muja?

The conversation is discussing the future of voice AI, specifically how it will impact the way humans interact with technology. The speaker is saying that in the future, voice AI will be so advanced that it will be able to understand and respond to human questions and commands, just like a human would. The speaker is also saying that this technology will be used in various applications, such as customer service, healthcare, and education.
